<a href="https://colab.research.google.com/github/GoncNeto/Data-Science/blob/master/Desafio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importar Libraries**

In [0]:
import pandas as pd
import io
from google.colab import files
from sklearn import tree
from sklearn import datasets, linear_model
from sklearn import model_selection
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

**Importar Dataset - Movies**

In [2]:
uploaded = files.upload()

#Load da database
movies_db = pd.read_csv(io.StringIO(uploaded['movie_metadata.csv'].decode('utf-8')))

Saving movie_metadata.csv to movie_metadata (9).csv


**Importar Dataset - Production Company**

In [3]:
uploaded = files.upload()

#Load da database
production = pd.read_csv(io.StringIO(uploaded['imdb_company.csv'].decode('utf-8')))

Saving imdb_company.csv to imdb_company (5).csv


**Merge dataframes:**

In [0]:
movies_db= movies_db.merge(production, on="movie_title", left_index=True, right_index=True)

**Tratamento de dados**

>**One Hot Encoding**

In [0]:
#eliminar as features, depois de ser feito o one hot encoding
movies_db = pd.concat([movies_db,pd.get_dummies(movies_db['director_name'],prefix='')],axis=1).drop(['director_name'],axis=1)
movies_db = pd.concat([movies_db,pd.get_dummies(movies_db['actor_2_name'],prefix='')],axis=1).drop(['actor_2_name'],axis=1)
movies_db = pd.concat([movies_db,pd.get_dummies(movies_db['color'],prefix='')],axis=1).drop(['color'],axis=1)
movies_db = pd.concat([movies_db,pd.get_dummies(movies_db['genres'],prefix='')],axis=1).drop(['genres'],axis=1)
movies_db = pd.concat([movies_db,pd.get_dummies(movies_db['country'],prefix='')],axis=1).drop(['country'],axis=1)

>**Label Encoding**

In [0]:
# Converter coluna para categoria
movies_db["actor_1_name"] = movies_db["actor_1_name"].astype('category')
movies_db["movie_title"] = movies_db["movie_title"].astype('category')
movies_db["actor_3_name"] = movies_db["actor_3_name"].astype('category')
movies_db["plot_keywords"] = movies_db["plot_keywords"].astype('category')
movies_db["language"] = movies_db["language"].astype('category')
movies_db["movie_imdb_link"] = movies_db["movie_imdb_link"].astype('category')
movies_db["content_rating"] = movies_db["content_rating"].astype('category')
movies_db["production"] = movies_db["production"].astype('category')

# Atribuir a variável codificada a nova coluna
movies_db["actor_1_name"] = movies_db["actor_1_name"].cat.codes
movies_db["movie_title"] = movies_db["movie_title"].cat.codes
movies_db["actor_3_name"] = movies_db["actor_3_name"].cat.codes
movies_db["plot_keywords"] = movies_db["plot_keywords"].cat.codes
movies_db["language"] = movies_db["language"].cat.codes
movies_db["movie_imdb_link"] = movies_db["movie_imdb_link"].cat.codes
movies_db["content_rating"] = movies_db["content_rating"].cat.codes
movies_db["production"] = movies_db["production"].cat.codes

In [0]:
movies_db.dtypes

>**Preencher NaNs - Mean**

In [0]:
columns = ['actor_1_name', 'movie_title', 'actor_3_name', 'language', 'content_rating', 'budget', 'imdb_score','num_critic_for_reviews','duration','director_facebook_likes','actor_3_facebook_likes','actor_1_facebook_likes','gross','facenumber_in_poster','num_user_for_reviews','title_year','actor_2_facebook_likes','aspect_ratio','production']

from sklearn.preprocessing import Imputer
train= movies_db[columns]


imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(train)
train= imp.transform(train)

movies_db[columns] = train

>Verificar se existe algum NaN:

In [0]:
movies_db.isnull().sum() 

**Classificação do dataset**

>**Split training set/test set**

In [14]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

#Usar gross
target= movies_db['gross']
X = movies_db.drop('gross', axis = 1)
[features_train, features_test, classes_train, classes_test] = model_selection.train_test_split(X, target, test_size=0.20, random_state = 100) 

model = tree.DecisionTreeRegressor()
model.fit(features_train, classes_train)

score_train = model.score(features_train, classes_train) 
score_test = model.score(features_test, classes_test)

print("Decision Tree")
print("score_train:", score_train)
print("score_test:", score_test)
print("\n")

Decision Tree
score_train: 1.0
score_test: 0.32391334962012563




>Cross Validation

In [0]:
from sklearn.model_selection import cross_val_score

#%%time
estimador = 100
score_result = []

for i in range (4):
  rf = RandomForestRegressor(max_depth=42, random_state=0, n_estimators=estimator)
  scores = cross_val_score(rf, train_X, train_y, cv=5)
  print("Average value for "+ str(estimador) + " is : " + str(scores.mean()))
  score_result.append(scores.mean())
  estimador += 100
  
print(score_result)

>**Curva de Aprendizagem**

In [0]:
# Load libraries

import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import learning_curve
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

train_sizes = [1, 100, 500, 1500, 3000, 4000]

# Create CV training and test scores for various training set sizes
train_sizes, score_train, score_test = learning_curve(LinearRegression(), 
                                                        X, 
                                                        target,
                                                        # Number of folds in cross-validation
                                                        cv=10,
                                                        # Evaluation metric
                                                        scoring = 'mean_squared_error')

# Create CV training and test scores for various training set sizes
train_sizes2, score_train2, score_test2 = learning_curve(RandomForestRegressor(), 
                                                        X, 
                                                        target,
                                                        # Number of folds in cross-validation
                                                        cv=10,
                                                        # Evaluation metric
                                                        scoring = 'mean_squared_error')

>LinearRegression:

In [0]:
import numpy as np

# Create means and standard deviations of training set scores
train_mean = np.mean(score_train, axis=1)
train_std = np.std(score_train, axis=1)

# Create means and standard deviations of test set scores
test_mean = np.mean(score_test, axis=1)
test_std = np.std(score_test, axis=1)

# Draw lines
plt.plot(train_sizes, train_mean, '--', color="#111111",  label="Training score")
plt.plot(train_sizes, test_mean, color="#111111", label="Cross-validation score")

# Draw bands
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color="#DDDDDD")
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color="#DDDDDD")

# Create plot
plt.title("Learning Curve")
plt.xlabel("Training Set Size"), plt.ylabel("Accuracy Score"), plt.legend(loc="best")
plt.tight_layout()
plt.show()

>RandomForestRegressor:

In [0]:
# Create means and standard deviations of training set scores
train_mean2 = np.mean(score_train2, axis=1)
train_std2 = np.std(score_train2, axis=1)

# Create means and standard deviations of test set scores
test_mean2 = np.mean(score_test2, axis=1)
test_std2 = np.std(score_test2, axis=1)

# Draw lines
plt.plot(train_sizes2, train_mean2, '--', color="#111111",  label="Training score")
plt.plot(train_sizes2, test_mean2, color="#111111", label="Cross-validation score")

# Draw bands
plt.fill_between(train_sizes2, train_mean2 - train_std2, train_mean2 + train_std2, color="#DDDDDD")
plt.fill_between(train_sizes2, test_mean2 - test_std2, test_mean2 + test_std2, color="#DDDDDD")

# Create plot
plt.title("Learning Curve")
plt.xlabel("Training Set Size"), plt.ylabel("Accuracy Score"), plt.legend(loc="best")
plt.tight_layout()
plt.show()

>**Cross Validation**

In [0]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

columns = ['actor_1_name', 'movie_title', 'actor_3_name', 'language', 'content_rating', 'budget', 'imdb_score']

# use the list to select a subset of the DataFrame (X)
X = movies_db[columns]

# since we're selecting only one column, we can select the attribute using .attribute
y = movies_db.gross

# 10-fold cross-validation with all three features
# instantiate model
lm = LinearRegression()

# store scores in scores object
# we can't use accuracy as our evaluation metric since that's only relevant for classification problems
# RMSE is not directly available so we will use MSE
scores = cross_val_score(lm, X, y, cv=10, scoring='mean_squared_error')
print(scores)